In [4]:
import carla
import time
from intersection_scenario_manager import IntersectionScenario
import numpy as np
import sys
sys.path.append("cartesian_to_frenet/")
from topology_extraction import get_opendrive_tree, get_junction_topology, get_junction_roads_topology
sys.path.append("../../global_route_planner/")
from global_planner import get_client, draw_waypoints, spawn_vehicle

import shapely
from shapely.geometry import Point, LineString, MultiLineString

In [5]:
# Get client to interact with CARLA server
client = get_client()

# Get current CARLA world
world = client.get_world()

all_waypoints = world.get_map().generate_waypoints(1)
def draw(road_id, lane_id=None, color=[0,255,0]):
    for wp in all_waypoints:
        
        if lane_id == None and wp.road_id == road_id:
            draw_waypoints(world, [wp], 10, color) 
#             print(wp.lane_id)
        elif wp.road_id == road_id and wp.lane_id==lane_id:
            draw_waypoints(world, [wp], 10, color) 
            
def filter_waypoints(waypoints, road_id, lane_id = None):
    
    filtered_waypoints = []
    for wp in waypoints:
        
        if lane_id == None and wp.road_id == road_id:
            filtered_waypoints.append(wp) 
        elif wp.road_id == road_id and wp.lane_id==lane_id:
            filtered_waypoints.append(wp) 

    return filtered_waypoints
            
tree = get_opendrive_tree(world)
junction_topology = get_junction_topology(tree)
road_topology = get_junction_roads_topology(tree)


Connection to CARLA server established!


In [6]:
SM = IntersectionScenario(client)

Intersection Manager Initialized...


dict_keys([224, 965, 421, 1175, 905, 1162, 139, 1260, 685, 334, 751, 1148, 1050, 53, 599, 1070, 943, 509, 924, 829, 245])

In [24]:
junctionId = 53

In [34]:
ego_vehicle, my_vehicles, incoming_road_lane_id_to_outgoing_lane_id_dict, intersection_topology, ego_key, global_path_wps, road_lane_to_orientation = SM.reset(num_vehicles=10, warm_start_duration=5, junction_id=junctionId)

generating scenario at junction id:  53
Control handed to system....


In [36]:
intersecting_left, intersecting_right, parallel_same_dir, parallel_opposite_dir = intersection_topology
ego_lane, parallel_same_dir = separate_ego_lane_and_other_parallel_lanes(parallel_same_dir, ego_key)

In [16]:
wps = get_lane_waypoints(all_waypoints, intersecting_right[1], road_lane_to_orientation)
ls = convert_to_shapely_linestring(wps)

wps1 = get_lane_waypoints(all_waypoints, ego_lane[0], road_lane_to_orientation)
ls2 = convert_to_shapely_linestring(wps1) 

In [17]:
intersection_point = ls.intersection(ls2)

In [21]:
draw_waypoints(world, wps[:10])

In [38]:
for element in parallel_same_dir:
    for item in element:
        draw(item[0], item[1])

In [35]:
ego_nearest_waypoint = client.get_world().get_map().get_waypoint(
    ego_vehicle.get_location(), project_to_road=True
)
client.get_world().debug.draw_string(
    ego_nearest_waypoint.transform.location,
    "O",
    draw_shadow=False,
    color=carla.Color(r=255, g=0, b=0),
    life_time=100,
)

In [39]:
ego_vehicle.destroy()
for actor in my_vehicles:
    actor.destroy()

In [10]:
def get_lane_waypoints(all_waypoints, road_lane_collection, road_lane_to_orientation):
    
    incoming_road = road_lane_collection[0][0]
    incoming_lane = road_lane_collection[0][1]
    
    connecting_road = road_lane_collection[1][0]
    connecting_lane = road_lane_collection[1][1]
    
    outgoing_road = road_lane_collection[2][0]
    outgoing_lane = road_lane_collection[2][1]
    
    incoming_waypoints = filter_waypoints(all_waypoints, incoming_road, incoming_lane)
    connecting_waypoints = filter_waypoints(all_waypoints, connecting_road, connecting_lane)
    outgoing_waypoints = filter_waypoints(all_waypoints, outgoing_road, outgoing_lane)
    
    if(road_lane_to_orientation[(incoming_road, incoming_lane)][-1] == 0): # 0 : Starting from near junction
        incoming_waypoints = incoming_waypoints[::-1]
        
    if(road_lane_to_orientation[(outgoing_road, outgoing_lane)][-1] == 1): # 1 : Ending at junction
        outgoing_waypoints = outgoing_waypoints[::-1]


    first_connecting_waypoint = connecting_waypoints[0]
    last_connecting_waypoint = connecting_waypoints[-1]
    
    last_incoming_waypoint = incoming_waypoints[-1]
    
    dist1 = first_connecting_waypoint.transform.location.distance(last_incoming_waypoint.transform.location)
    dist2 = last_connecting_waypoint.transform.location.distance(last_incoming_waypoint.transform.location)

    if(dist1 > dist2):
        connecting_waypoints = connecting_waypoints[::-1]
        
    
    return incoming_waypoints + connecting_waypoints[1:-1] + outgoing_waypoints
    


    
def convert_to_shapely_linestring(waypoints):
    
    list_of_coordinates = [Point(waypoint.transform.location.x, waypoint.transform.location.y) for waypoint in waypoints]
    linestring = LineString(list_of_coordinates)
    
    return linestring


def separate_ego_lane_and_other_parallel_lanes(parallel_same_dir, ego_key):
    
    ego_lane = []
    other_lanes = []
    
    for elem in parallel_same_dir:
        flag = 0
        for item in elem:
            if(item == ego_key):
                flag = 1
        if(flag == 1):
            ego_lane.append(elem)
        else:
            other_lanes.append(elem)
    return ego_lane, other_lanes
            
    
    